In [31]:
# import statements

import os,random, shutil
from imutils import paths
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adagrad
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

### Building Train, Validation and Test sets

In [20]:
# Declaring paths
INPUT_DATASET = "Dataset/archive"
BASE_PATH = "Dataset/new"
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1

In [ ]:
originalPaths=list(paths.list_images(INPUT_DATASET))
random.seed(7)
random.shuffle(originalPaths)
index=int(len(originalPaths)*TRAIN_SPLIT)
trainPaths=originalPaths[:index]
testPaths=originalPaths[index:]
index=int(len(trainPaths)*VAL_SPLIT)
valPaths=trainPaths[:index]
trainPaths=trainPaths[index:]
datasets=[("training", trainPaths, TRAIN_PATH),
          ("validation", valPaths, VAL_PATH),
          ("testing", testPaths, TEST_PATH)
]
for (setType, originalPaths, basePath) in datasets:
        print(f'Building {setType} set')
        if not os.path.exists(basePath):
                print(f'Building directory {basePath}')
                os.makedirs(basePath)
        for path in originalPaths:
                file=path.split(os.path.sep)[-1]
                label=file[-5:-4]
                labelPath=os.path.sep.join([basePath,label])
                if not os.path.exists(labelPath):
                        print(f'Building directory {labelPath}')
                        os.makedirs(labelPath)
                newPath=os.path.sep.join([labelPath, file])
                shutil.copy2(path, newPath)


### Building Cancer Classification Model

In [48]:
model=Sequential()
shape=(48,48,3)
channelDim=-1



model.add(SeparableConv2D(32, (3,3), padding="same",input_shape=shape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(SeparableConv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(SeparableConv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(SeparableConv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(SeparableConv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(SeparableConv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation("softmax"))

### Training Model

In [49]:
# Setting-up and Image Preprocessing
NUM_EPOCHS=5; INIT_LR=1e-2; BS=32

trainPaths=list(paths.list_images(TRAIN_PATH))
lenTrain=len(trainPaths)
lenVal=len(list(paths.list_images(VAL_PATH)))
lenTest=len(list(paths.list_images(TEST_PATH)))

trainLabels=[int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels=np_utils.to_categorical(trainLabels)
classTotals=trainLabels.sum(axis=0)
classWeight=classTotals.max()/classTotals
classWeight ={0:classWeight[0],1:classWeight[1]}

trainAug = ImageDataGenerator(
	rescale=1/255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

valAug=ImageDataGenerator(rescale=1 / 255.0)

trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
testGen = valAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)



Found 255684 images belonging to 2 classes.
Found 42588 images belonging to 2 classes.
Found 99938 images belonging to 2 classes.


In [50]:
# Compilation of model
opt=Adagrad(lr=INIT_LR,decay=INIT_LR/NUM_EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])

In [51]:
# Training model

trainedModel=model.fit_generator(
	trainGen,
	steps_per_epoch=lenTrain//BS,
	validation_data=valGen,
	validation_steps=lenVal//BS,
	class_weight=classWeight,
	epochs=NUM_EPOCHS)

Epoch 1/5


/tmp/ipykernel_52960/1439354680.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  trainedModel=model.fit_generator(


7990/7990 [==============================] - 1450s 181ms/step - loss: 0.6300 - accuracy: 0.8153 - val_loss: 0.6305 - val_accuracy: 0.7449
Epoch 2/5
7990/7990 [==============================] - 1302s 163ms/step - loss: 0.6056 - accuracy: 0.8215 - val_loss: 0.6112 - val_accuracy: 0.7474
Epoch 3/5
7990/7990 [==============================] - 1329s 166ms/step - loss: 0.6014 - accuracy: 0.8218 - val_loss: 0.6136 - val_accuracy: 0.7479
Epoch 4/5
7990/7990 [==============================] - 1336s 167ms/step - loss: 0.5982 - accuracy: 0.8225 - val_loss: 0.6193 - val_accuracy: 0.7451
Epoch 5/5
7990/7990 [==============================] - 1315s 165ms/step - loss: 0.5982 - accuracy: 0.8229 - val_loss: 0.6362 - val_accuracy: 0.7366


In [59]:
print("Now evaluating the model")
testGen.reset()
pred_indices=model.predict_generator(testGen,steps=(lenTest//BS)+1)

pred_indices=np.argmax(pred_indices,axis=1)

print(classification_report(testGen.classes, pred_indices, target_names=testGen.class_indices.keys()))

cm=confusion_matrix(testGen.classes,pred_indices)
total=sum(sum(cm))
accuracy=(cm[0,0]+cm[1,1])/total
specificity=cm[1,1]/(cm[1,0]+cm[1,1])
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
print(cm)
print(f'Accuracy: {accuracy}')
print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')



Now evaluating the model


/tmp/ipykernel_52960/3131523291.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred_indices=model.predict_generator(testGen,steps=(lenTest//BS)+1)


              precision    recall  f1-score   support

           0       0.96      0.66      0.78     71544
           1       0.52      0.93      0.67     28394

    accuracy                           0.74     99938
   macro avg       0.74      0.80      0.73     99938
weighted avg       0.84      0.74      0.75     99938

[[47083 24461]
 [ 1849 26545]]
Accuracy: 0.736736776801617
Specificity: 0.9348806085792774
Sensitivity: 0.6580985128033099


In [ ]:
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N), trainedModel.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), trainedModel.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), trainedModel.history["accuracy"], label="train_acc")
plt.plot(np.arange(0,N), trainedModel.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on the Dataset")
plt.xlabel("Epoch No.")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show('LossAccPlot.png')